In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, types, text
from datetime import datetime, timedelta

# Crear conexión con SQL Server usando autenticación de Windows
engine = create_engine('mssql+pyodbc://BIANCA\\SQLEXPRESS/PremiumDrinks?trusted_connection=yes&driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes')

# Crear un rango de fechas para tu calendario (por ejemplo, de 2016 a 2024)
inicio_fecha = datetime(2016, 1, 1)
fin_fecha = datetime(2024, 12, 31)
fechas = pd.date_range(start=inicio_fecha, end=fin_fecha)

# Crear el DataFrame de calendario
calendario_df = pd.DataFrame({
    'FECHA': fechas,
    'Año': fechas.year,
    'Mes': fechas.month,
    'Día': fechas.day,
    'Semana': fechas.isocalendar().week,
    'EsFestivo': np.where((fechas.month == 12) & (fechas.day == 25), 1, 0)  # Ejemplo: solo marca el 25 de diciembre como festivo
})

# Puedes agregar más días festivos según tu región, ajustando las condiciones en 'EsFestivo'
# Añadamos más ejemplos de festivos (ajusta según el país):
festivos = [
    '2024-01-01',  # Año Nuevo
    '2024-12-25',  # Navidad
    # Añade más fechas festivas
]
calendario_df['EsFestivo'] = calendario_df['FECHA'].isin(pd.to_datetime(festivos)).astype(int)

# Verificar y eliminar duplicados (aunque no debería haber)
calendario_df = calendario_df.drop_duplicates(subset=['FECHA']).dropna(subset=['FECHA'])

# Especificar los tipos de datos para cada columna en SQL Server
dtype_sql_calendario = {
    'FECHA': types.DATE,
    'Año': types.INTEGER,
    'Mes': types.INTEGER,
    'Día': types.INTEGER,
    'Semana': types.INTEGER,
    'EsFestivo': types.INTEGER
}

# Exportar el DataFrame de calendario a una tabla SQL llamada CALENDARIO
calendario_df.to_sql('CALENDARIO', con=engine, if_exists='replace', index=False, dtype=dtype_sql_calendario)

print("Tabla CALENDARIO creada exitosamente con los datos de fechas.")

with engine.connect() as connection:
    # Asegurarse de que la columna FECHA no tenga valores nulos y sea NOT NULL
    connection.execute(text("""
        ALTER TABLE CALENDARIO
        ALTER COLUMN FECHA DATE NOT NULL;
    """))

    # Agregar clave primaria a la tabla CALENDARIO
    connection.execute(text("""
        ALTER TABLE CALENDARIO
        ADD CONSTRAINT PK_FECHA PRIMARY KEY (FECHA);
    """))

print("Clave primaria agregada exitosamente a la tabla CALENDARIO.")

Tabla CALENDARIO creada exitosamente con los datos de fechas.
Clave primaria agregada exitosamente a la tabla CALENDARIO.
